# 基于MindNLP的风格化聊天助手

## 环境配置

1. 配置python3.9环境


In [1]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [2]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

***注：以上代码执行完成后，需点击左上角或右上角将kernel更换为python-3.9.0***

2. 安装mindspore2.2.14，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)
3. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)
    这里用的是mindnlp-0.3.2

In [1]:
# %%capture captured_output

!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.2.14/MindSpore/unified/x86_64/mindspore-2.2.14-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple
# !wget https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/mindnlp_install/mindnlp-0.3.2-py3-none-any.whl
!pip install mindnlp-0.3.2-py3-none-any.whl
!pip install gradio mdtex2html -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install ipywidgets -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 743.0/743.0 MB 4.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 79.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.3/309.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 84.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 76.1 MB/s eta 0:00:0000:0100:01
Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
Processing ./mindnlp-0.3.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 80.7

In [2]:
%%capture captured_output
!wget -P /home/ma-user/work https://openi.pcl.ac.cn/lvyufeng/frpc-gradio/raw/branch/master/frpc_linux_amd64
!cp /home/ma-user/work/frpc_linux_amd64 /home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2
!chmod +x /home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/gradio/frpc_linux_amd64_v0.2

***注：执行如上命令完成安装后，请点击上方的restart kernel图标重启kernel，再进行实验***

### 数据集下载与处理

1. 数据集加载

    本次使用的数据集是自建数据集，由《西游记》原著截取汇总所得，共含有1316条数据。
    样例：  
    {  
        "que_state": "那樵汉慌忙丢了斧，转身答礼",  
        "question": "不敢当人！不敢当人！我这个粗人衣食都不足，怎么敢称自己为“神仙”呢？",  
        "ans_state": "",  
        "answer": "你不是神仙，如何说出神仙的话来？"  
    },

2. 数据预处理  
    将文件用gb18030编码打开，以question为输入文本，以answer为输出文本。

In [3]:
import json
import random
def process_dataset(source, batch_size=32, shuffle=False):

    column_names = ['input_ids', 'attention_mask','labels','text_labels']
    
    dataset = GeneratorDataset(source, column_names=column_names, shuffle=shuffle)
    
    dataset = dataset.batch(batch_size)

    return dataset

class MSDataset:
    def __init__(self, filepath,tokenizer,max_length):
        self.path = filepath
        self.instruction = []
        self.input_sentence= []
        self.output_sentence = []
        self._load()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _load(self):
        
        with open(self.path, encoding="gb18030") as f:
            data = json.load(f)
            for d in data:
                self.instruction.append(d["question"])
                self.output_sentence.append(d["answer"])

    def __getitem__(self, index):
        inputs = self.instruction[index]
        output_sentence = self.output_sentence[index]
        model_inputs = self.tokenizer(inputs, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="np")
        labels = self.tokenizer(output_sentence, max_length=self.max_length, padding="max_length", truncation=True, return_tensors="np")
        labels = labels["input_ids"]
        labels[labels == self.tokenizer.pad_token_id] = -100
        
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels, output_sentence

    def __len__(self):
        return len(self.instruction)

3.  加载tokenizer  
    AutoTokenizer用于加载ChatGLM-6B的tokenizer；  

In [4]:
from mindnlp.transformers import AutoTokenizer, ChatGLM2Tokenizer

tokenizer_name_or_path = "ZhipuAI/ChatGLM-6B"#"stylellm/SanGuoYanYi-6b"
# tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path,mirror="modelscope")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path,mirror="modelscope",revision='master')
len(tokenizer)

130344

4. 处理数据

In [5]:
from mindspore.dataset import GeneratorDataset
max_length = 128
batch_size = 8
dataset = process_dataset(MSDataset("./XiYouJi/XiYouJi.json",tokenizer,max_length),batch_size=batch_size)
train_dataset, eval_dataset = dataset.split([0.9, 0.1])

In [6]:
print(next(train_dataset.create_dict_iterator()))

{'input_ids': Tensor(shape=[8, 1, 128], dtype=Int64, value=
[[[     3,      3,      3 ...     31, 130001, 130004]],
 [[     3,      3,      3 ...  63823, 130001, 130004]],
 [[     3,      3,      3 ...     31, 130001, 130004]],
 ...
 [[     3,      3,      3 ...     31, 130001, 130004]],
 [[     3,      3,      3 ...  63823, 130001, 130004]],
 [[     3,      3,      3 ...     35, 130001, 130004]]]), 'attention_mask': Tensor(shape=[8, 1, 1, 128, 128], dtype=Bool, value=
[[[[[ True,  True,  True ...  True,  True,  True],
    [ True,  True,  True ...  True,  True,  True],
    [ True,  True,  True ...  True,  True,  True],
    ...
    [ True,  True,  True ... False, False,  True],
    [ True,  True,  True ... False, False,  True],
    [ True,  True,  True ... False, False, False]]]],
 [[[[ True,  True,  True ...  True,  True,  True],
    [ True,  True,  True ...  True,  True,  True],
    [ True,  True,  True ...  True,  True,  True],
    ...
    [ True,  True,  True ... False, False,  True

### 模型构建

1. 构建ChatGLM-6B模型，并配置采用IA3微调的模型

In [2]:
# creating model
from mindnlp.transformers import  AutoModelForCausalLM, ChatGLM2ForConditionalGeneration, AutoModelForSeq2SeqLM

model_name_or_path = "ZhipuAI/ChatGLM-6B"#"stylellm/SanGuoYanYi-6b"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path, mirror="modelscope").half()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
from mindnlp.peft import get_peft_model, IA3Config, TaskType, LoraConfig

peft_config = IA3Config(task_type=TaskType.CAUSAL_LM, inference_mode=False,target_cells=["query_key_value"], feedforward_cells=[])#,target_cells=["query_key_value"], feedforward_cells=[]
# peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
#加载模型并打印微调参数量，可以看到仅有不到0.0056%的参数参与了微调。

trainable params: 344,064 || all params: 6,173,630,464 || trainable%: 0.005573122686988218


### 模型训练
先定义训练的epoch, lr和优化器等；
后按照如下步骤定义训练逻辑：

1. 构建正向计算函数
2. 函数变换，获取微分函数
3. 定义训练一个step的逻辑
4. 遍历训练数据集进行模型训练，同时每一个epoch后，打印loss

In [7]:
num_epochs = 30
learning_rate = 8e-3
power = 0.9

num_training_steps = num_epochs * len(train_dataset)

In [8]:
# optimizer and lr scheduler
from mindspore import nn
from mindspore.nn.learning_rate_schedule import PolynomialDecayLR

lr_scheduler=PolynomialDecayLR(learning_rate,learning_rate*0.001,num_training_steps,power)
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)


In [9]:
# training and evaluation
from mindspore import Tensor
import mindspore
from tqdm import tqdm
import numpy as np

num_batches = len(train_dataset)
num_batches_eval = len(eval_dataset)
                       
def forward_fn(input_ids,attention_mask,labels ):
        output = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
        )
        return output.loss, output.logits
    
grad_fn = mindspore.value_and_grad(
        forward_fn, None,optimizer.parameters, has_aux=True
    )
for epoch in range(num_epochs):
    model.set_train(True)
    total_loss, total_step = 0, 0
    correct = 0
    total = 0
    with tqdm(total=num_batches) as t:
        for step, (input_ids,attention_mask,labels,_) in enumerate(train_dataset):
            input_ids  = input_ids.squeeze(axis=1)
            labels  = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            (loss, logits), grad = grad_fn(input_ids,attention_mask,labels)
            optimizer(grad)
            total_loss += loss.asnumpy()
            total_step += 1
            curr_loss = total_loss / total_step
            t.set_postfix({'train-loss': f'{curr_loss:.2f}'})
            t.update(1)
    model.set_train(False)
    eval_loss = 0
    total_step = 0
    eval_preds = []
    text_labels = []
    with tqdm(total=num_batches_eval) as t:
        for step, (input_ids,attention_mask,labels,text) in enumerate(eval_dataset):
            input_ids = input_ids.squeeze(axis=1)
            labels = labels.squeeze(axis=1)
            attention_mask = attention_mask.squeeze(axis=1)
            outputs = model(input_ids=input_ids,attention_mask=attention_mask,labels=labels)
            loss = outputs.loss
            eval_loss += loss.asnumpy()
            total_step += 1           
            eval_loss = total_loss / total_step
            eval_preds.extend(
                tokenizer.batch_decode(np.argmax(outputs.logits.asnumpy(), -1), skip_special_tokens=True)
            )
            text_str = str(text.asnumpy())
            text_str = text_str.replace("[", "").replace("]", "").replace(",", "").replace("'", "")
            labels = text_str.split(" ")
            text_labels.extend(labels)
            t.set_postfix({'eval-loss': f'{eval_loss:.2f}'})
            t.update(1)
    for pred, text_label in zip(eval_preds, text_labels):
        if pred.strip() == text_label.strip():
            correct += 1
        total += 1
    # accuracy = correct / total * 100
    # print(f"{accuracy=} % on the evaluation dataset")
    eval_epoch_loss = eval_loss / eval_dataset.get_dataset_size()
    eval_ppl = np.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / train_dataset.get_dataset_size()
    train_ppl = np.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 16/16 [00:17<00:00,  1.12s/it, eval-loss=67.40] 


epoch=0: train_ppl=1390.8974819752664 train_epoch_loss=7.237704488255034 eval_ppl=67.5298816094073 eval_epoch_loss=4.2125701904296875


100%|██████████| 16/16 [00:17<00:00,  1.11s/it, eval-loss=61.57]


epoch=1: train_ppl=743.4600386019812 train_epoch_loss=6.611315016778524 eval_ppl=46.898793180371996 eval_epoch_loss=3.847991943359375


100%|██████████| 16/16 [00:17<00:00,  1.11s/it, eval-loss=60.65]


...

epoch=27: train_ppl=49.317721473640596 train_epoch_loss=3.898283478397651 eval_ppl=9.66897983324001 eval_epoch_loss=2.268922805786133


100%|██████████| 16/16 [00:17<00:00,  1.11s/it, eval-loss=35.62]


epoch=28: train_ppl=45.82335397226184 train_epoch_loss=3.8247938732172817 eval_ppl=9.26412634505702 eval_epoch_loss=2.226149559020996


100%|██████████| 16/16 [00:17<00:00,  1.12s/it, eval-loss=35.32]

epoch=29: train_ppl=44.39114559232113 train_epoch_loss=3.7930400259542787 eval_ppl=9.094481741976914 eval_epoch_loss=2.207667827606201


### 微调模型推理

1. 保存训练后的微调模型权重

In [4]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)
# save_checkpoint(model, os.path.join(peft_model_id, "adapter_model.ckpt"))

In [5]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

1.4M	ZhipuAI/ChatGLM-6B_IA3_CAUSAL_LM/adapter_model.ckpt


2. 加载保存的模型权重

In [9]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

3. 在测试集上进行推理

In [13]:
import datasets
text_column = "inputs"
model.set_train(False)

i = 13
def load_dataset(filepath):
    inputs = []
    labels = []
    with open(filepath, encoding="gb18030") as f:
        data = json.load(f)
        for d in data:
            inputs.append(d["question"])
            labels.append(d["answer"])
    
    dataset = datasets.Dataset.from_dict({
        "inputs": inputs,
        "label": labels,
    })
    return dataset

dataset = load_dataset("./XiYouJi/XiYouJi.json")
dataset = dataset.train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

inputs = tokenizer(dataset["validation"][text_column][i], return_tensors="ms")
print(dataset["validation"][text_column][i])
print(inputs)

outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=128)
# outputs,_ = model.chat(tokenizer,query=dataset["validation"][text_column][i], max_length=128)
print(outputs)
text_output = tokenizer.batch_decode(outputs ,skip_special_tokens=True)
print(text_output)

哥哥啊，你又来捣乱了！用麻绳捆住我，还喷水让我放松，你这个瘦子可能不觉得什么，我这个胖子可受不了啊！不信你看，我的两个胳膊上，绳子已经勒进肉里两寸深了，怎么逃脱呢？
{'input_ids': Tensor(shape=[1, 59], dtype=Int64, value=
[[     5,  67891,  63991 ...     31, 130001, 130004]])}


/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1561: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


[[     5  67891  63991      6  83319  63860 112108  63828     35  63864
   66205  69401  82802  64505  63838      6  63893 100692  64682  66544
       6  88236  66856  63914  64124  87009  64061      6  80114  78960
   63879  75113  63991     35 123417  63922      6  64237  64413  75796
   63839      6  80877  63943  66629  64177  64417  63884  64054  67593
   64478  63828      6  64473  84159  63911     31 130001 130004      5
  123641      6  70270  68241  75890      6  98116  63863  82802  64505
   63967  64667 120045  69298  67864  63823  63838  84775  64542  70414
   63826  64091  79727  64086  63823  64157  65402  64330  66318  64370
   97930  90657  63853  67864      6 122280  70998  63826  75640  63823
   75989  79642 111528  64471      6  64157  67305  67696      6  66794
   72888  84207  64542  63823 130005]]
['哥哥啊,你又来捣乱了!用麻绳捆住我,还喷水让我放松,你这个瘦子可能不觉得什么,我这个胖子可受不了啊!不信你看,我的两个胳膊上,绳子已经勒进肉里两寸深了,怎么逃脱呢? 很抱歉,作为一个人工智能助手,我无法被捆住或者受到任何形式的暴力威胁。我的目的是帮助解决问题和提供有用的信息。请确保您的行为不会对他人造成伤害或威胁,尊重他人的权利和尊

### 基于RAG技术的聊天机器人开发
（1）使用检索增强生成技术，借助百度对回答进行增强  
（2）使用爬虫技术，对百度百科页面进行解析，获取对应部分的内容  
（3）调取聚合数据API，实时获取任意城市的天气情况  
（4）针对不同形式的问题，确定是否使用检索 

In [15]:
%%capture captured_output
!pip install baidusearch
!pip install requests beautifulsoup4

In [16]:
import os
from typing import Dict, Union, Optional
from typing import List


import mdtex2html

#将文本中的字符转为网页上可以支持的字符，避免被误认为是HTML标签
def parse_text(text):
    """copy from https://github.com/GaiZhenbiao/ChuanhuChatGPT/"""
    lines = text.split("\n")
    lines = [line for line in lines if line != ""]
    count = 0
    for i, line in enumerate(lines):
        if "```" in line:
            count += 1
            items = line.split('`')
            if count % 2 == 1:
                lines[i] = f'<pre><code class="language-{items[-1]}">'
            else:
                lines[i] = f'<br></code></pre>'
        else:
            if i > 0:
                if count % 2 == 1:
                    line = line.replace("`", "\`")
                    line = line.replace("<", "&lt;")
                    line = line.replace(">", "&gt;")
                    line = line.replace(" ", "&nbsp;")
                    line = line.replace("*", "&ast;")
                    line = line.replace("_", "&lowbar;")
                    line = line.replace("-", "&#45;")
                    line = line.replace(".", "&#46;")
                    line = line.replace("!", "&#33;")
                    line = line.replace("(", "&#40;")
                    line = line.replace(")", "&#41;")
                    line = line.replace("$", "&#36;")
                lines[i] = "<br>"+line
    text = "".join(lines)
    return text

In [17]:
from mindnlp.transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import gradio as gr
from baidusearch.baidusearch import search
import requests
from bs4 import BeautifulSoup


def online_search(query):
    query += " 百度百科"
    try:
        results = search(query, num_results=5)
        if results:
            result = results[0]
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
            }
            url = result['url']
            response = requests.get(url, headers=headers, allow_redirects=True)
            actual_url = response.url
            response = requests.get(actual_url, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            description_meta = soup.find('meta', attrs={'name': 'description'})
            if description_meta:
                content = description_meta.get('content')
                if content:
                    sentences = content.split('。')
                    first_two_sentences = '。'.join(sentences[:2]) + '。'
                    outputs,_ = model.chat(tokenizer, query = first_two_sentences, max_length = 128)
                    return outputs
                else:
                    return "未找到description的内容"
            else:
                return "未找到description标签"
        else:
            return "未找到相关信息。"
    except Exception as e:
        return f"搜索服务出现错误：{str(e)}"


def get_weather(city):
    api_key = 'e34030e335b8b08801cf05f769cf42aa' 
    weather_url = f"http://apis.juhe.cn/simpleWeather/query?city={city}&key={api_key}"
    response = requests.get(weather_url)
    if response.status_code == 200:
        data = response.json()
        if data['error_code'] == 0:
            result = data['result']
            weather = result['realtime']
            return f"{city}的天气情况：\n温度：{weather['temperature']}℃\n湿度：{weather['humidity']}\n天气：{weather['info']}\n风向：{weather['direct']}"
        else:
            return f"获取天气信息失败：{data['reason']}"
    else:
        return "无法连接到天气服务。"

#采用流聊天方式（stream_chat）调用模型，使得生成答案有逐字生成的效果
def predict_chatglm(input, chatbot, max_length, top_p, temperature, history):
    chatbot.append((parse_text(input), ""))
    for response, history in model.stream_chat(tokenizer, input, history, max_length=max_length, top_p=top_p,
                                               temperature=temperature):
        chatbot[-1] = (parse_text(input), parse_text(response))       

        yield chatbot, history
        
def generate_answer(input_text,history):
    # 检查是否是天气查询
    if '天气' in input_text:
        city = input_text.split('天气')[0].strip()
        weather_info = get_weather(city)
        return weather_info
    elif '?' in input_text or '？' in input_text:
        outputs,_ = model.chat(tokenizer, query = input_text, max_length = 128)
        return outputs
    else:
        return online_search(input_text)
            
# 创建Gradio界面
def predict(input_text, history):
    answer = generate_answer(input_text,history)
    history.append((input_text, answer))
    return history, history

with gr.Blocks() as demo:
    gr.HTML("""<h1 align="center">MindNLP ChatGLM-6B with RAG</h1>""")
    chatbot = gr.Chatbot()
    with gr.Row():
        with gr.Column(scale=4):
            user_input = gr.Textbox(show_label=False, placeholder="Input...", lines=3, container=False)
            submitBtn = gr.Button("Ask", variant="primary")
            emptyBtn = gr.Button("Clear")
        with gr.Column(scale=1):
            max_length = gr.Slider(0, 4096, value=2048, step=1.0, label="Maximum length", interactive=True)
            top_p = gr.Slider(0, 1, value=0.7, step=0.01, label="Top P", interactive=True)
            temperature = gr.Slider(0, 1, value=0.95, step=0.01, label="Temperature", interactive=True)               

    history = gr.State([])

    submitBtn.click(predict, [user_input, history], [chatbot, history])
    submitBtn.click(lambda: "", None, user_input)
    emptyBtn.click(lambda: ([], []), None, [chatbot, history])

demo.queue().launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://f84d2269a7c766ae21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/ma-user/anaconda3/envs/python-3.9.0/lib/python3.9/site-packages/mindnlp/transformers/generation/utils.py:1561: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://hf-mirror.com/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


#### 基于GPT-SoVITS的文本转语音

利用开源项目GPT-SoVITS的api，实现zero-shot的文本转模拟人声。